## Random Forest: middle ground between interpretability and accuracy?

#### Table of Contents

1. [Introduction](#introduction)
2. [Simulation Study](#simulation_study)
    1. [Data Analysis](#data_analysis)
    2. [Simulation](#simulation)
    3. [Simulation Results](#simulation_results)
3. [Empirical Application](#empirical_application)
4. [References](#References)

In [ ]:
install.packages("pROC")
install.packages("dplyr")
install.packages("splitstackshape")
install.packages("moments")
install.packages("sn")

library(rpart)
library(randomForest)
library(pROC)
library(dplyr)
library(splitstackshape) 
library(moments)
library(sn)

### 1. Introduction <a class="anchor" id="introduction"></a>


Decision Tree can be a very useful tool when the structure of the data analysed is known to be highly non-linear and complex. It has also an advantage of simplicity of explanation: decision tree can be displayed graphically and is easy to interpret. The last point is particularly important when the results of an analysis are used by people less familiar with regression and classification techniques. The significant disadvantage of the method is the lack of predictive accuracy when compared to other methods like linear regressions, support vector maschines and unsupervised learning techniques. 

<img src="random_forest.png" width=1000 height=1000 style="float:center" />

image source: https://towardsdatascience.com/random-forests-and-decision-trees-from-scratch-in-python-3e4fa5ae4249

Random Forest is an approach that is based on a Decision Tree method and aimed to increase prediction accuracy. It is based on several decorrelated decision trees, each trained on a different random sub-sample (e.g. bootstrapped training samples). The prediction from the Random Forest is then an average of predictions of the decision trees in case of regression or the class with the highest occurence in case of classification. The way decorrelation of the trees is achieved is by allowing to consider only subset of the features at each split of a tree, with each subset being randomly chosen. This means that at some splits it will not be possible to use the strongest predictors. Corespondingly, the model's prediction will be less dependent on a subsample of the strongest features and therefore will be less volatile. This approach leads to lower variance and as a result to higher predictive power. The higher predictive power, however, comes at the cost of lower interpretability as simple graph of a single Decision Tree and intuitive explanation based on that are no more available.

<img src="balance.png" width=500 height= 500 />

image source: https://mc.ai/interpretability-vs-accuracy-the-friction-that-defines-deep-learning/

It is important to notice that even though interpretability of the Random Forest is lower when compared to the Decision Tree, it remains high relative to such methods as support vector maschine (SVM), neural network, etc. The latter is basically a black box, and therfore much less interpretable and requires additional analysis to get any information other than predictions themselves. It is possible, for example, to retrieve an information about the features' importance using Gini index or residual sum of squares in case of Random Forest (this functionality often comes in the same programming package), whereas additional method (like Decision Tree) is needed to get any kind of additional information in the case of Neural Network or SVM. However, just like in the case of Decision Tree and Random Forest, Random Forest being more interpretable in most of the cases will lack acuracy precision when compared to the methods mentioned.   

Thus, in the case of non-linear and complex structure of data, the choice between these methods is nothing else than a choice between the interpretability and prediction precision. The right choice of balance is highly dependent on the goal of an analysis and the nature of data in question. Therefore for the appropriate data and goal, the Random Forest can be the golden middle between the prediction accuracy of Neural Network, SVM and the interpretability of Decision Tree.  

For this project I use data that I believe to be suitable for the Random Forest method, namely, the data from direct marketing campaigns of a Portuguese retail bank. Unfortunately, this is not exactly the same data set (some of the features were not included due to the privacy concerns) that was used in the paper this project based on: "A data-driven approach to predict the success of bank telemarketing" (Moro S., Cortez P. and Rita P., 2014). The paper is available at https://www.sciencedirect.com/science/article/abs/pii/S016792361400061X .The dataset is public and available for research at http://archive.ics.uci.edu/ml/datasets/Bank+Marketing# .

### 2. Simulation Study <a class="anchor" id="simulation_study"></a>

Telemarketing campaign is a method of reaching out to potential customers remotely with the goal of convincing to buy a product or service. In our case it is a phone calls to potential clients of a Portugese retail bank with the goal of convincing to open a deposit sell. This method is one of the most widely used as it allows to contact a large number of people, but in order to increase the effeciency further only those customers that are the most likely to purchase a product should be contacted. To achieve this managers often use Decision Support Systems, software-based programmes processing large amounts of data in order to help businesses and organisations in the decision-making process. Data mining is one of the techniques that is widely used, with the most frequent task being classification. In the current study the question is to determine whether the client is likely or not to open a deposit sell based on the personal characteristics as well as available information about current economic situation. However, this is not the only goal of the analysis, the main determinants of the result are also of interest for the managers since they will help to create an appropriate policy and in doing so affect future client behaviour. The goal is, therefore, to construct a model that will have a high level of accuracy as well as interpretability. This is the task that is often faced and that poses a question of an apropriate balance of accuracy and interpretability and, accordingly, model. 

Authors of the paper are considerng following models for this task: Logistic Regression, Decision Tree, Support Vector Machine (SVM) and Neural Networks. While their analysis shows that the first two of the model lack predictive power, it also becomes apparent that the last two models are hard to interpret. In particular the model that achieved the best accuracy, Neural Networks, is in an essence a black box. To retrieve some of the information additional tools were used such as Data-based Sensitivity Analysis algorithm and Decision Tree applied to the output responses of the fitted model (S.Moro, et al, 2014). In this project I analyse an alternative method that I believe, given the data in question, could be a golden middle between accuracy of the Neural Networks, SVM and the interpretability of Logistic regression and Decision Tree.  

There are two reasons I believe this data is appropriate for the Random Forest analysis

1. There is a high chance that the structure of data is non-linear and complex. This is supported by the results of the analysis performed in the paper, where the lowest predictive accuracy was achieved by Logistic regression, even when compared to Decision Tree which is known to have a low predictive power on the one hand and a good fit for a non-linear, complex structure on the other. These two considerations led me to believe that underlying structure is non-linear. Moreover, from the data analysis multicollinearity problem becomes apparent and while it affects Random Forest analysis, the situation for any linear regression is much worse. Therefore Random Forest is more preferable that any other approach based on a linear regression such as Logistic Regression.

2. The nature of the data. The approach taken by authors of the paper is to achieve a high reaisticity by constracting a rolling window evaluation. In the reality, the new information becomes available regularly and at the same time the behaviour of the client (and the probability of consuming a product) changes with the time following changes in the environment. I keep this approach and construct my version of the "rolling window". I believe that Random Forest is more suitable when compared to the Decision Tree, as it will be more efffecient in incorporating new information to make a better prediction.


The main goal of the analysis that follows is to show that Random Forest offers a considerable improvement in the prediction accuracy when compared to the Decision Tree. I do not consider Logistic Regression for now since it performed poorly when compared to the Decision Tree. I also do not consider SVM and Neural Networks since the idea is that Random Forest outperforms them in terms of interpretability and not accuracy (I assume that both of these techniques will achieve a higher predictive accuracy). To demonstrate a higher interpretability, I will apply "in-built" fuctions using which one can easily access, for example,  the feature importance data.  

#### 2.A Data analysis <a name="data_analysis"></a>

Upon closer inspection of the data, it becomes apparent that it differs considerbly from the data used in the study by Moro S., Cortez P. and Rita P. (2014) It affects the study and most importantly results that are no more comparable to those ones presented in the paper. 

In what follows I retrieve the data information that will be used later on in the simulation study, whenever necessary I add comments directly to the code.

In [ ]:
# Download the data and do some cleaning
df <- read.csv("bank-additional-full.csv", sep=";")
df[df == "unknown"] <- NA
df <- na.omit(df)
df$duration <- NULL # information related to the last call (the dependent variable in the setting)
df$y[df$y == "yes"] <- 1
df$y[df$y == "no"] <- 0
df$y <- as.numeric(df$y)
df$pdays[df$pdays == 999] <- 0 # 999 means that client was not previously contacted
vector.quali <- c('job', 'marital', 'education', 'default', 'housing', 'loan', 'contact',
                  'month', 'day_of_week', 'poutcome', 'y')
df[vector.quali] <- lapply(df[vector.quali], factor)
df[c("age", "campaign", "pdays", "previous")] <- sapply(df[c("age", "campaign", "pdays", "previous")],as.numeric)


In [ ]:
# Get fraction (in percentage) of each categorical variable
x <- lapply(df[, c('job', 'marital', 'education', 'default', 'housing', 'loan', 'contact',
                       'month', 'day_of_week', 'poutcome', 'y')], table)

neat.table <- function(x, name){
  xx <- data.frame(x)
  names(xx) <- c("Value", "Count")
  xx$Fraction <- with(xx, Count*100/sum(Count))
  data.frame(Variable = name, xx)
}

do.call(rbind, lapply(seq_along(x), function(i)neat.table(x[i], names(x[i]))))

# information on "time-invariant" continuous variables: age, campaign, pdays, previous
hist(df$previous)
hist(df$age)
hist(df$campaign)
hist(df$pdays)

The data contains both categorical and continuous variables. As can be seen from the graphs above some of the continuous variables have skewed distributions. The variables can be divided further into two subgroups: personal information variables (e.g. profession, information on whether person does or does not have a loan, housing mortgage, etc.) and variables describing economic environment at that point of time (e.g. number of employees, consumer confidence index, etc.). The dependent variable is a boolean: yes/no response to wether the person has decided to open a deposit sell. In the next subsection I describe my approach to data simulation.

#### 2.B Simulation <a name="simulation"></a>

I subdevided data simulation in 4 parts according to the nature of the data and challanges it presents. 

1. Time-dependent variables 

    The analysis of Moro et al. (2014) shows the economic and social indicators are the main determinants of the response variable. It is important to notice that these are also the variables that tend to change with the time (as opposed to, for example, profession). And as was stated before this is one of the reasons I believe Random Forest will perform better when compared to the Decision Tree. Hence, it was important to incorporate the time trend into the simulation. In order to make simulation as realistic as possible I decided to use the real-world data on economic indicators of that period of time.  These features are:
 * emp.var.rate: employment variation rate - quarterly indicator 
 * cons.price.idx: consumer price index - monthly indicator 
 * cons.conf.idx: consumer confidence index - monthly indicator 
 * euribor3m: euribor 3 month rate - daily indicator 
 * nr.employed: number of employees - quarterly indicator 
 
   I retrieved them from the data and in order to make all quarters equally represented performed some data manipulation. Unfortunately, there is no information about the time period when each call was made, the only available information is that the data ordered by date (from May 2008 to November 2010). The way I attempted to overcome this obstacle is: after choosing all unique combinations of above mentioned variables, I have randomly chosen seven data points corresponding  to each value of a mothly indicator (consumer confidence index). Out of these I have randomly chosen 7 data points corresponding to each value of the quarterly indicator (number of employees).This left me with 77 observations (7 multiplied by 11, number of quarters). In the simulated data each data point will be represented equally and therfore there is a requirement for the total number of observations to be a factor of 77. The choice of the monthly indicator variable was arbitrary since both monthly indicators contain a unique value corresponding to each month. Quarterly indicator variable wasn't an arbitrary choice since number of employees indicator was the only variable containing a unique value for each quarter. The number 7 was chosen because it is the minimum number of unique data points per each monthly indicator and as I tried to achieve the higher data variation among daily, monthly and quarterly indicators, it did not to make sense to choose value lower than that (choosing the value higher on the other hand would have resulted in the unequal representation of some quarters). In addition to the variables mentioned above I have also added a corresponding month as it is directly related to the values of the monthly indicators and creat a high level of collinearity which I also wanted to include in my simulated data in order for it to be more representative of the real-world data.
   
2. Categorical variables

    The way I simulated categorical variables (with the exception of month) is relatively straightforward: I gathered information on the fraction of each response in the original data and used these to represent probabilities of getting these responses in the simulated data.
    
3. Continuous variables

    As was pointed out before continuous variable have a skewed distributions and taking logarithms unfortunately did not help much. To simulate these variables I used an sn R package, which allowed me to specify the skeweness and kurtosis of the original distributions. This, however, also posed some challenges as there exist boundaries for possible combinations of these two values. The feasibilty region is nicely represented in the fig. 1 in the paper by Arellano-Valle R. B. and Azzalini A. (2013) "The centred parameterization and related quantities of the skew-t distribution" (on a picture below: $\gamma_1$ is a skeweness and $\gamma_2$ is a kurtosis). Where it was impossible to use the original values I used the closest feasible value.
    
    <img src="kurtosis.jpg" width=400 height= 400 />

    image source: Arellano-Valle R. B. and Azzalini A. (2013) "The centred parameterization and related quantities of the skew-t distribution"
    
4. Response variable
    
    To simulate a response variable I decided to utilize the relationship within the original data set. I grow a Classification Tree on the original data and then apply it to the explanatory variables in the simulated data set. As a result I obtain probabilities of the output being a success (i.e. "yes"), to which I add some noise (random draw from normal distribution with mean 0 and standard deviation of 0.2). Finally, I classify a simulated response variable to be success if the resulted probability is larger than a certain threshold ("prob.succcess" in the function below, default is 0.5). For the comparison reasons I add a variable y.true, which is assigned success if the probability of it without noise added is higher than 0.5. 
    This approach has a considerable drawbacks as person looses control over the structure of the simulated dataset, but it helps to achieve a goal of giving a non-linear structure and makes simulated data closer to the real-world processes.

In [ ]:
"""
Fuction for the data simulation.
 
 input:  N (integer): total number of observations to be simulated; should be factor of 77.
         dataset (dataframe): original data set.
         prob.success (numerical): probability threshold for the response variable determening success/failure.
         
 output: dataframe: simulated data set with additional columns of y.prob.true (true probability of success), 
                    y.true (assigned success, i.e 1, if y.prob.true > 0.5), e (error rate), y.prob (sum of y.prob.true and error rate).
 
"""

df.simulation <- function(N, dataset, prob.success=0.5){
  
  # "time-invariant" variables
  # categorical:
  job <- sample(c("housemaid", "services", "admin.", "technician", "blue-collar", "unemployed", "retired", 
                  "entrepreneur", "management", "student", "self-employed"), N, replace=TRUE, 
                prob=c(0.027, 0.09, 0.287, 0.1795, 0.186, 0.024, 0.0399, 0.0357, 0.0758, 0.02, 0.0358))
  marital <- sample(c("divorced", "married", "single"), N, replace=TRUE, prob=c(0.1165, 0.5737, 0.3097))
  education <- sample(c("basic.4y", "basic.6y", "basic.9y", "high.school", "illiterate", 
                        "professional.course", "university.degree"), N, replace=TRUE, 
                      prob=c(0.078, 0.046, 0.14, 0.25, 0.0004, 0.14, 0.342))
  default <- sample(c("no", "yes"), N, replace=TRUE, prob=c(99.99, 0.01))
  housing <- sample(c("no", "yes"), N, replace=TRUE, prob=c(45.8, 54.2))
  loan <- sample(c("no", "yes"), N, replace=TRUE, prob=c(84.36, 15.64))
  contact <- sample(c("cellular", "telephone"), N, replace=TRUE, prob=c(67.05, 32.95))
  day_of_week <- sample(c("fri", "mon", "thu", "tue", "wed"), N, replace=TRUE, prob=c(0.188, 0.206, 0.209, 
                                                                                      0.195, 0.2))
  poutcome <- sample(c("failure", "nonexistent", "success"), N, replace=TRUE, prob=c(0.114, 0.847, 0.39))
  #numerical:
  age.params <- cp2dp(c(mean(dataset$age), sd(dataset$age), skewness(dataset$age)), "SN")
  age <- c(rsn(N, dp=age.params))
  campaign.params <- cp2dp(c(mean(dataset$age), sd(dataset$age), skewness(dataset$age), 
                             kurtosis(dataset$campaign)), "ST")
  campaign <- c(rst(N, dp=campaign.params))
  pdays.params <- cp2dp(c(mean(dataset$pdays), sd(dataset$pdays), 3.3, 
                             kurtosis(dataset$pdays)), "ST")
  pdays <- c(rst(N, dp=pdays.params))
  previous.params <- cp2dp(c(mean(dataset$previous), sd(dataset$previous), 2.5, 
                             kurtosis(dataset$previous)), "ST")
  previous <- c(rst(N, dp=previous.params))
 
  #create dataframe out of "time-invariant" variables
  df.nottime.var <- cbind(job, marital, education, default, housing, loan, contact, day_of_week, 
                          poutcome, age, campaign, pdays, previous)
  colnames(df.nottime.var) <- c("job", "marital", "education", "default", "housing", "loan",
                                "contact", "day_of_week", "poutcome", "age", "campaign", 
                                "pdays", "previous")
  df.nottime.var <- as.data.frame(df.nottime.var)
  
  # time-dependent variables
  # Create a data frame which contains time-dependent variables. 
  # The values are chosen randomly  for each quarter from the original data set. 
  # Each quarter contains the same number of people interviewed (factor of 7).
  df.time.unique <- unique(dataset[,c('month', 'emp.var.rate','nr.employed','cons.conf.idx','cons.price.idx','euribor3m')])
  df.time.strat <- stratified(stratified(df.time.unique, "cons.conf.idx", 7), "nr.employed", 7)
  n.per.time <- N/nrow(df.time.strat)
  df.time.var <- df.time.strat[rep(seq_len(nrow(df.time.strat)), each = n.per.time), ]
  
  # dataframe of all explanatory variables
  df.full <- cbind(df.nottime.var, df.time.var)
  df.full$age <- as.integer(df.full$age)
  df.full$campaign <- as.integer(df.full$campaign)
  df.full$pdays <- as.integer(df.full$pdays)
  df.full$previous <- as.integer(df.full$previous)
  
  # use the whole empirical dataset to create tree and produce y.true for the simulated data set.
  tree.fit <- rpart(y~., data=dataset, method="class")
  y.predict.matrix <- predict(tree.fit, newdata=df.full, type="prob")
  df.full$y.prob.true <- as.vector(y.predict.matrix[,2])
  df.full$y.true <- as.factor(ifelse(df.full$y.prob.true > 0.5, 1, 0))
  df.full$e <- rnorm(n=N, mean=0, sd=0.2)
  df.full$y.prob <- df.full$y.prob.true + df.full$e
  df.full$y <- as.factor(ifelse(df.full$y.prob > prob.success, 1, 0))

  return(df.full)
  
}

As was noted before there are two points I want to address in the data simulation part. The one that is related to the non-linear structure of the data was incorporated in the data simulation process directly. The second point is conderned with the flow of the data and will be addressed in the evaluation part of the analysis. Moro S. et al. (2014) created a rolling window evaluation scheme performing several updates and discarding old data. The scheme consists of the following steps (see figure below):
1. A training window of consecutive observations of size W (i.e. training sample) is used to train model  
2. The next K observations are used to test the model
3. All K observations used for testing in step (2.) are added to the training model, while the first K observations are discarded
4. The process repeats until there are no observations left to create a new test set.

I implemeted the same scheme in the function below. I use Area Under the Curve (AUC) to evaluate the prediction accuracy of the Decision Tree and Random Forest. It is possible to choose the percentage of the data to be used for test set and to choose the number of runs (data updates) to be performed. The average AUCs are then calculated and returned in the form of a table.

<img src="rolling_window.png" width=500 height= 500 />

image source: S. Moro, P. Cortez and P. Rita. A Data-Driven Approach to Predict the Success of Bank Telemarketing (2014) 

In [ ]:
"""
Function for calculating  test Area Under the Curve (AUC) scores 
of Decision Tree and Random Forest applied to the simulated data set
using a rolling window evaluation. 

input: dataset (dataframe): simulated data set
       K (integer): number of runs, i.e. training and test sets updates
       test.size (numerical): fraction of the data used as a test set; 0 < test.size < 1

output: dataframe: results for the average test Area Under the Curve (AUC)

"""
sim.rolling.window.evaluation <- function(dataset, K, test.size){
  
  # calculate sizes of training and test sets
  r <- (1-test.size)/test.size
  N <- nrow(dataset) 
  n.train <- N*r/(K+r)
  n.test <- (N-n.train)/ K  
  
  # additional data needed for simulation evaluation
  y.true <- dataset$y.true
  y.true.train <- head(x=y.true, n=n.train)
  y.true.test.all <- tail(x=y.true, n=(N-n.train))
  y.true.test <- head(x=y.true.test.all,n=n.test)
  # drop unnecessary columns
  dataset$y.prob.true <- NULL
  dataset$e <- NULL
  dataset$y.prob.fin <- NULL
  dataset$y.true <- NULL
  
  # create: initial training set (train),
  #         set from which test sets are drawn (test.all), 
  #         first test set  (test)
  
  train <- head(x=dataset, n=n.train)
  y.train <- train$y
  predictors.train <- train[, names(train)!= "y"]
  test.all <- tail(x=dataset, n=(N-n.train))
  test <- head(x=test.all,n=n.test)
  y.test <- test$y
  predictors.test <- test[, names(test)!= "y"]
  
  
  # loop for 1) fitting each model using a train set, calculating respective test AUC 
  #          2) updating train and test data sets
  #          3) calculating test AUC
  
  AUC.tree.appended <- c()
  AUC.forest.appended <- c()
  
  for (i in 2:K){
    
    forest.fit <- randomForest(x=predictors.train, y=y.train)
    y.matrix.forest <- predict(object=forest.fit, newdata=predictors.test, type="prob")
    y.data.forest <- cbind.data.frame(as.vector(y.matrix.forest[,2]), y.true.test)
    AUC.forest.result <- auc(data=y.data.forest, response=y.data[,2], predictor=y.data[,1])
    AUC.forest.appended <- append(AUC.forest.appended, AUC.forest.result)
    
    tree.fit <- rpart(y~., data=train, method="class")
    y.matrix.tree <- predict(tree.fit, newdata=test, type="prob")
    y.data.tree <- cbind.data.frame(as.vector(y.matrix.tree[,2]), y.true.test)
    AUC.tree.result <- auc(data=y.data.tree, response=y.data[,2], predictor=y.data[,1])
    AUC.tree.appended <- append(AUC.tree.appended, AUC.tree.result)
    
    if (nrow(test.all) >= n.test){
      
      train <- tail(train, -n.test)
      train <- rbind(train, test)
      test.all <- tail(test.all, -n.test)
      test <- head(x=test.all,n=n.test)
      y.train <- train$y
      predictors.train <- train[, names(train)!= "y"]
      y.test <- test$y
      predictors.test <- test[, names(test)!= "y"]
      
      y.true.test.all <- tail(y.true.test.all, -n.test)
      y.true.test <- head(x=y.true.test.all,n=n.test)
      
    }
    
  }
  
  ave.AUC.forest <- mean(AUC.forest.appended)
  ave.AUC.tree <- mean(AUC.tree.appended)
  
  df.AUC <- data.frame(matrix(ncol=2, nrow=2))
  colnames(df.AUC) <- c("Method","ave.AUC")
  df.AUC$Method[1] <- "Decision Tree"
  df.AUC$Method[2] <- "Random Forest"
  df.AUC$ave.AUC[1] <- ave.AUC.tree
  df.AUC$ave.AUC[2] <- ave.AUC.forest
  
  return(df.AUC)
  
}

#### 2.C Simulation Results <a name="simulation_results"></a>

This subsections presents results that one can obtain using the simulation and evaluation functions described in the previous subsection and discusses the possible reasons for the large discrepency of the results obtained in this study and the ones obtained by authors of the paper.

In [ ]:
# Simulate data set
sim.data <- df.simulation(N=7700, dataset=df)

# Get the evaluation results
sim.results <- sim.rolling.window.evaluation(df.trial, 10, 0.10)
print(sim.results)

# remove unnecessary columns for the further analysis
sim.data$y.prob.true <- NULL
sim.data$e <- NULL
sim.data$y.prob.fin <- NULL
sim.data$y.true <- NULL

# fit Classification tree and get summary
tree.fit <- rpart(y~., data=sim.data, method="class")
summary(tree.fit)
plot(tree.fit)
text(tree.fit)

#check for multicollinearity of a subset of features
df.col <- data.frame(sim.data$emp.var.rate, sim.data$euribor3m, sim.data$nr.employed)
df.col[] <- lapply(df.col, as.numeric)  
chisq <- chisq.test(table(unlist(df.col)))
print(chisq)

df.col.add <- data.frame(sim.data$emp.var.rate, sim.data$euribor3m, sim.data$nr.employed, sim.data$cons.conf.idx,  sim.data$cons.price.idx)
df.col.add[] <- lapply(df.col.add, as.numeric)  
chisq.add <- chisq.test(table(unlist(df.col.add)))
print(chisq.add)

I obtain the same high prediction accuracy results for both, Decision Tree and Random Forest. This was not expected since accoring to the results presented in the paper Decision Tree performed poorly when compared to the other techniques, obtaining AUC of 0.757. Moreover, even the method that performed the best in case of Moro S., et al (2014), i.e. Neural Networks, achieved AUC of only 0.794. However, upon the closer inspection of the data, it becomes apparent that the majority of the features are different in two datasets, the one used in this study and the one used by the authors of the paper. Thus, some of the most important features such as direction of the call and the number of days since the agent login was created are absent in the data analysed. Moreover, only 4 of the most important features according to the Moro S. et al (2014) are presented (euribor 3 month rate, employment variation rate, number of employees, month of the call). As one can see from the summary of the Classification tree fit, all 4 are indeed among 8 most important variables. With 3 of them being economic indicators and the forth being a month in which those indicators were recorded and the cal was made, the chances of multicollinearity are high. This is confirmed by the chi-square test shown above. Another two variables among the most important ones are economic indicators as well. I performed chi-square test for all economic indicators included in the list of the most important variables and once again got a high statistically significant result. Only two of the most important outcomes are not economic indicators and are not related to the rest of features due to the way the data was simulated. Thus, due to the multicollinearity, it was possible to pick one or some of the economic indicators and achieve a good prediction accuracy (hence, small Classification Tree is grown). I believe this simplified task when compared to the classification problem on a data set used by Moro S. et al (2014). Since Classification Tree achieves a high accuracy rate, there is no room for Random Forest to outperform it. In this case Decision Tree, being simple to interpret and achieving a high accuracy rate, is of course an obvious choice for the task. However, this result should be taken with the grain of salt as the data set that was considered in this project is clearly not representative of the real-world data used by Moro S. et al (2014) and the similarity of the results of Decision Tree and Random Forest should make one question the correctness of the implemetation. 

Due to the way data was simulated it was expected that most of the difference between Decsion Tree and Random Forest would have come from the rolling window evaluation. It was expected that Random Forest will outperform Decision Tree in terms of accuracy prediction as test data sets contain new information from the days, months and quarters ahead of the training set. Decision Tree is not suited for making predictions based on out-of-sample values of features and therfore should have performed worse. This, however, was not supported by the results of the study. One of possible reasons is that differents between daily/mothly/quarterly indicators was not high enough from sample to sample to qualify for a "ou-of-sample values". 


### 3. Empirical application <a class="anchor" id="empirical_application"></a>


Since the simulation study was essentialy based on the original data similar results are expected. For this section an additional function for rolling window evaluation was created with the sole difference being a comparison of predicted class to the the one given in data instead of the simulated true response. 

In [ ]:
"""
Function for calculating  test Area Under the Curve (AUC) scores 
of Decision Tree and Random Forest applied to the original data set
using a rolling window evaluation. 

input: dataset (dataframe): data set
       K (integer): number of runs, i.e. training and test sets updates
       test.size (numerical): fraction of the data used as a test set; 0 < test.size < 1

output: dataframe: results for the average test Area Under the Curve (AUC)

"""
rolling.window.evaluation <- function(dataset, K, test.size){
  
  # calculate size of a training set (n.train)
  r <- (1-test.size)/test.size
  N <- nrow(dataset) 
  n.train <- N*r/(K+r)
  
  
  # create: initial training set (train),
  #         set from which test sets are drawn (test.all), 
  #         first test set  (test)
  
  train <- head(x=dataset, n=n.train)
  y.train <- train$y
  predictors.train <- train[, names(train)!= "y"]
  test.all <- tail(x=dataset, n=(N-n.train))
  n.test <- nrow(test.all)/ K  # size of each test set
  test <- head(x=test.all,n=n.test)
  y.test <- test$y
  predictors.test <- test[, names(test)!= "y"]
  
  
  # loop for 1) fitting each model using a train set, calculating respective test AUC 
  #          2) updating train and test data sets
  #          3) calculating test AUC
  
  AUC.tree.appended <- c()
  AUC.forest.appended <- c()
  
  for (i in 2:K){
    
    forest.fit <- randomForest(x=predictors.train, y=y.train)
    y.matrix.forest <- predict(object=forest.fit, newdata=predictors.test, type="prob")
    y.data.forest <- cbind.data.frame(as.vector(y.matrix.forest[,2]), y.test)
    AUC.forest.result <- auc(data=y.data.forest, response=y.data[,2], predictor=y.data[,1])
    AUC.forest.appended <- append(AUC.forest.appended, AUC.forest.result)
    
    tree.fit <- rpart(y~., data=train, method="class")
    y.matrix.tree <- predict(tree.fit, newdata=test, type="prob")
    y.data.tree <- cbind.data.frame(as.vector(y.matrix.tree[,2]), y.test)
    AUC.tree.result <- auc(data=y.data.tree, response=y.data[,2], predictor=y.data[,1])
    AUC.tree.appended <- append(AUC.tree.appended, AUC.tree.result)
    
    if (nrow(test.all) >= n.test){
      
      train <- tail(train, -n.test)
      train <- rbind(train, test)
      test.all <- tail(test.all, -n.test)
      test <- head(x=test.all,n=n.test)
      y.train <- train$y
      predictors.train <- train[, names(train)!= "y"]
      y.test <- test$y
      predictors.test <- test[, names(test)!= "y"]
      
    }
    
  }
  
  ave.AUC.forest <- mean(AUC.forest.appended)
  ave.AUC.tree <- mean(AUC.tree.appended)
  
  df.AUC <- data.frame(matrix(ncol=2, nrow=2))
  colnames(df.AUC) <- c("Method","ave.AUC")
  df.AUC$Method[1] <- "Decision Tree"
  df.AUC$Method[2] <- "Random Forest"
  df.AUC$ave.AUC[1] <- ave.AUC.tree
  df.AUC$ave.AUC[2] <- ave.AUC.forest
  
  return(df.AUC)
  
}

In [ ]:
# get rolling window evaluation results
emp.results <- rolling.window.evaluation(dataset=df,K=10, test.size = 0.10)
print(emp.results)

# fit Classification tree and get summary
emp.tree.fit <- rpart(y~., data=df, method="class")
summary(emp.tree.fit)
plot(emp.fit.tree)
text(emp.fit.tree)

# fit Random Forest
df.response <- df$y
df$y <- NULL
forest.fit <- randomForest(x=df, y=df.response)
importance(forest.fit)
varImpPlot(forest.fit)

As expected after simulation analysis Random Forest and Decision Tree show the same result. For comparison reasons the summary of fitted Decision Tree is also included. Given the current data set it makes more sense to use Decision Tree as it offers both high interpreatability and prediction accuracy. The aim of this project was to show that for some data sets Random Forest could be a good balance of prediction accuracy and interpretability, and even though the first part with regards to prediction accuracy did not work out well,I still decided to demonstrate relative interpretability of Random Forest. I fitted the model for the whole set and printed the model information that includes the most important features and the graph for better visualisation. 

### 4. References <a class="anchor" id="refernces"></a>

* Arellano-Valle R. B. and Azzalini A. The centred parameterization and related quantities of the skew-t distribution. Journal of Multivariate Analysis, Elsevier, 113: 73-90 January 2013
* S. Moro, P. Cortez and P. Rita. A Data-Driven Approach to Predict the Success of Bank Telemarketing. Decision Support Systems, Elsevier, 62:22-31, June 2014
* [Moro et al., 2014] S. Moro, P. Cortez and P. Rita. A Data-Driven Approach to Predict the Success of Bank Telemarketing. Decision Support Systems, Elsevier, 62:22-31, June 2014